In [8]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [9]:
import pandas as pd

def charger_csv(path, nom_indicateur):
    df = pd.read_csv(path, skiprows=3, sep=";", names=["Période", nom_indicateur, "Code"])
    df = df.drop(columns=["Code"])
    return df

# Chargement
df1 = charger_csv("input/serie_001565530_19052025/valeurs_mensuelles.csv", "Climat_Affaires")
df2 = charger_csv("input/serie_001688527_19052025/valeurs_trimestrielles.csv", "Taux_Chomage")
df3 = charger_csv("input/serie_001769682_19052025/valeurs_mensuelles.csv", "Indice_Prix_Conso")
df4 = charger_csv("input/serie_010768261_19052025/valeurs_mensuelles.csv", "Indicateur_production_indus")

def expand_trimestres(df):
    trimestre_to_mois = {
        "T1": ["01", "02", "03"],
        "T2": ["04", "05", "06"],
        "T3": ["07", "08", "09"],
        "T4": ["10", "11", "12"]
    }
    lignes_expandees = []
    for _, row in df.iterrows():
        periode = row["Période"]
        valeur = row["Taux_Chomage"]
        if "T" in periode:
            annee, trim = periode.split("-")
            for mois in trimestre_to_mois[trim]:
                lignes_expandees.append({
                    "Période": f"{annee}-{mois}",
                    "Taux_Chomage": valeur
                })
        else:
            lignes_expandees.append({
                "Période": periode,
                "Taux_Chomage": valeur
            })
    return pd.DataFrame(lignes_expandees)


df2_expanded = expand_trimestres(df2)

# Fusion
df_merged = df1.merge(df2_expanded, on="Période", how="outer") \
               .merge(df3, on="Période", how="outer") \
               .merge(df4, on="Période", how="outer")

# Tri
df_merged = df_merged.sort_values("Période").reset_index(drop=True)

# Affichage
df_merged.tail(10)

,Période,Climat_Affaires,Taux_Chomage,Indice_Prix_Conso,Indicateur_production_indus
595,2024-08,96.8,7.4,120.26,101.77
596,2024-09,98.0,7.4,119.88,101.41
597,2024-10,97.5,7.3,120.09,101.16
598,2024-11,96.1,7.3,120.20,101.09
599,2024-12,94.5,7.3,120.48,100.88
600,2025-01,95.1,7.4,121.02,100.32
601,2025-02,95.7,7.4,120.36,101.32
602,2025-03,96.6,7.4,120.18,101.49
603,2025-04,96.4,NaN,120.57,NaN
604,Période,NaN,NaN,NaN,NaN


In [10]:
df_merged.shape

(605, 5)

In [11]:
# Chargement du fichier (sauter les 6 lignes de métadonnées)
df_oblig = pd.read_csv("input/WEBSTAT-observations-2025-05-19T23_11_39.031+02_00.csv",
                       skiprows=6,
                       sep=";",
                       names=["Date", "Taux_10ans_FR"])

# Nettoyage
df_oblig["Date"] = pd.to_datetime(df_oblig["Date"], errors="coerce")
df_oblig["Taux_10ans_FR"] = df_oblig["Taux_10ans_FR"].str.replace(",", ".", regex=False)
df_oblig["Taux_10ans_FR"] = pd.to_numeric(df_oblig["Taux_10ans_FR"], errors="coerce")

# Création d'une colonne 'Période' au format YYYY-MM
df_oblig["Période"] = df_oblig["Date"].dt.strftime("%Y-%m")

# On garde uniquement les colonnes utiles
df_oblig = df_oblig[["Période", "Taux_10ans_FR"]]

# Fusion avec le dataframe existant
df_merged_final = df_merged.merge(df_oblig, on="Période", how="outer")

# Tri et réinitialisation de l'index
df_merged_final = df_merged_final.sort_values("Période").reset_index(drop=True)

# Export si besoin
df_merged_final.to_csv("donnees_mergees_complet.csv", index=False, encoding="utf-8-sig")

# Aperçu final
df_merged_final.tail(10)

,Période,Climat_Affaires,Taux_Chomage,Indice_Prix_Conso,Indicateur_production_indus,Taux_10ans_FR
595,2024-08,96.8,7.4,120.26,101.77,2.9681
596,2024-09,98.0,7.4,119.88,101.41,2.9126
597,2024-10,97.5,7.3,120.09,101.16,2.9983
598,2024-11,96.1,7.3,120.20,101.09,3.0860
599,2024-12,94.5,7.3,120.48,100.88,3.0164
600,2025-01,95.1,7.4,121.02,100.32,3.3141
601,2025-02,95.7,7.4,120.36,101.32,3.1526
602,2025-03,96.6,7.4,120.18,101.49,3.4784
603,2025-04,96.4,NaN,120.57,NaN,3.2953
604,Période,NaN,NaN,NaN,NaN,NaN
